In [41]:
import pandas as pd
import urllib
import lxml
import time

In [ ]:
abstracts = pd.DataFrame()

import xml.etree.ElementTree as ET


rxns = pd.read_csv("../Data_CSVs/brenda_rxns_incomplete.csv")

tot = rxns.pubmedId.unique().size

In [46]:
rxns.head()

,Unnamed: 0,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber,pubmedId
0,0,NaN,695811.0,Thermus sp.,1% activity compared to cyclohexanone,?,NaN,? + NAD+,NaN,(-)-carvone + NADH + H+,1.1.1.1,18704396.0
1,1,NaN,695811.0,Thermus sp.,125% activity compared to cyclohexanol,r,NaN,(rac)-3-methylcyclohexanone + NADH + H+,163% activity compared to cyclohexanone,"(1S,3S)-3-methylcyclohexanol + NAD+",1.1.1.1,18704396.0
2,2,NaN,735498.0,Yokenella sp.,"41.5% of the activity with (2E)-but-2-enal, yi...",?,NaN,(2E)-2-methylpent-2-en-1-ol + NADP+,NaN,(2E)-2-methylpent-2-enal + NADPH + H+,1.1.1.1,24509923.0
3,3,NaN,735498.0,Yokenella sp.,"37.4% of the activity with (2E)-but-2-enal, yi...",?,NaN,"(2E)-3,7-dimethylocta-2,6-dien-1-ol + NADP+",NaN,"(2E)-3,7-dimethylocta-2,6-dienal + NADPH + H+",1.1.1.1,24509923.0
4,4,NaN,691867.0,Homo sapiens,substrate of isozyme ADH4,r,NaN,"(2E)-non-2-ene-1,4-diol + NAD+",NaN,(2E)-4-hydroxynon-2-enal + NADH + H+,1.1.1.1,18505683.0


In [49]:
rxns_abstract = rxns[["literatureSubstrates","ecNumber","substrates","products", "pubmedId"]]



In [75]:
cache = {}
count = 0
t0 = time.time()
for pmid in rxns_abstract.pubmedId.unique():
    if (count % 100) == 0:
        print(str(count) + " out of " + str(tot))
        print(time.time() - t0)
        print("\n")
    count = count + 1
    try:
        #get the abstract yay!
        t1 = time.time()
        req =  urllib.request.Request("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&api_key=f1ae75aea73f93cb7061e61b596138d0fa08&retmode=xml&id={}".format(pmid)) # this will make the method "POST"
        resp = urllib.request.urlopen(req).read().decode("utf-8") 
        tree = ET.fromstring(resp)
        tree = list(tree)[0]
        abstract_f = tree.find(".//AbstractText")
        if abstract_f is not None:
            cache[pmid] = abstract_f.text
        else:
            if "AbstractText" in resp:
                cache[pmid] = "UNCLEAN" + resp.split("AbstractText")[1]
            else:
                continue
    except Exception as e:
        print(e)
        print(pmid)
        continue
    

0 out of 103948
0.012671232223510742


5.7220458984375e-05
0.9327573776245117
0.00067901611328125
2.5987625122070312e-05
5.817413330078125e-05
1.02783203125
0.0007162094116210938
2.574920654296875e-05
6.4849853515625e-05
0.6198117733001709
0.0003466606140136719
2.3126602172851562e-05
5.91278076171875e-05
HTTP Error 400: Bad Request
0.0
7.796287536621094e-05
0.4024670124053955
0.0004620552062988281
2.4080276489257812e-05
7.82012939453125e-05
0.732274055480957
0.0008039474487304688
3.790855407714844e-05
0.00010418891906738281
0.46997499465942383
0.0006577968597412109
2.09808349609375e-05
6.699562072753906e-05
0.5056991577148438
0.0007219314575195312
2.8133392333984375e-05
8.916854858398438e-05
0.47151899337768555
0.0006399154663085938
3.0040740966796875e-05
9.894371032714844e-05
0.5863990783691406
0.0003190040588378906
2.193450927734375e-05
7.081031799316406e-05
0.4888288974761963
0.0003437995910644531
2.3603439331054688e-05
7.581710815429688e-05
0.5167529582977295
0.0004420280456542969


KeyboardInterrupt: 

In [ ]:
rxns_abstract['abstract'] = rxns_abstract['pubmedId'].map(cache)


In [71]:
cache

{18704396.0: 'The alcohol dehydrogenase from Thermus sp. ATN1 (TADH) was characterized biochemically with respect to its potential as a biocatalyst for organic synthesis. TADH is a NAD(H)-dependent enzyme and shows a very broad substrate spectrum producing exclusively the (S)-enantiomer in high enantiomeric excess (>99%) during asymmetric reduction of ketones. TADH is active in the presence of 10% (v/v) water-miscible solvents like 2-propanol or acetone, which permits the use of these solvents as sacrificial substrates in substrate-coupled cofactor regeneration approaches. Furthermore, the presence of a second phase of a water-insoluble solvent like hexane or octane had only minor effects on the enzyme, which retained 80% of its activity, allowing the use of these solvents in aqueous/organic mixtures to increase the availability of low-water soluble substrates. A further activity of TADH, the production of carboxylic acids by dismutation of aldehydes, was investigated. This reaction us

In [77]:
tot/2

51974.0